<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Le *broadcasting*

In [ ]:
import numpy as np

## Complément - niveau intermédiaire

Lorsqu'on a parlé de programmation vectorielle, on a vu qu'on pouvait écrire quelque chose comme ceci :

In [ ]:
X = np.linspace(0, 2 * np.pi)
Y = np.cos(X) + np.sin(X) + 2

Je vous fais remarquer que dans cette dernière ligne on combine
* deux tableaux de mêmes tailles  - quand on ajoute `np.cos(X)` avec `np.sin(X)`,
* ou un tableau avec un scalaire - quand on ajoute `2` au résultat.

En fait le broadcasting est ce qui permet 
* d'unifier le sens de ces deux opérations
* et même de donner du sens à des cas plus généraux, ou on fait des opérations entre des tableaux qui ont des *tailles différentes* mais assez semblables pour qu'on puisse tout de même les combiner.

## Exemples en 2D

Nous allons commencer par quelques exemples simples, avant de généraliser le mécanisme. Nous nous donnons pour commencer un tableau de base :

In [ ]:
a = 100 * np.ones((3, 5), dtype=np.int32); print(a)

Je vais illustrer le broadcasting avec l'opération `+`, mais bien entendu ce mécanisme est à l'œuvre dès que vous faites des opérations entre deux tableaux qui n'ont pas les mêmes dimensions.

Pour commencer je vais donc ajouter à mon tableau de base un scalaire :

### Broadcasting entre les dimensions `(3, 5)` et `(1,)`

In [ ]:
print(a)

In [ ]:
b = 3
print(b)

***

Lorsque j'ajoute ces deux tableaux, c'est comme si j'avais ajouté à `a` la différence:

In [ ]:
# pour élaborer c
c = a + b
print(c)

In [ ]:
# c'est comme si j'avais
# ajouté à a ce terme-ci
print(c - a)

C'est un premier cas particulier de *broadcasting* dans sa version extrême.

Le scalaire `b`, qui est en l'occurrence considéré comme un tableau dont le `shape` vaut `(1,)`, est dupliqué dans les deux directions jusqu'à obtenir ce tableau uniforme de taille `(5, 3)` et qui contient un `3` partout.

Et c'est ce tableau, qui est maintenant de la même taille que `a`, qui est ajouté à `a`.

Je précise que cette explication est du domaine du modèle pédagogique; je ne dis pas que l'implémentation va réellement allouer un second tableau, bien évidemment on peut optimiser pour éviter cette construction inutile.

### Broadcasting `(3, 5)` et `(5)`

Voyons maintenant un cas un peu moins évident. Je peux ajouter à mon tableau de base un tableau de taille `(1, 5)`. Voyons cela :

In [ ]:
print(a)

In [ ]:
b = np.arange(1, 6); print(b)

In [ ]:
b.shape

****

Ici encore, je peux ajouter les deux termes :

In [ ]:
# je peux ici encore
# ajouter les tableaux
c = a + b
print(c)

In [ ]:
# et c'est comme si j'avais
# ajouté à a ce terme-ci
print(c - a)

Avec le même point de vue que tout à l'heure, on peut se dire qu'on a d'abord transformé (broadcasté) le tableau `b` : 

depuis la dimension `(5,)`

vers la dimension `(3, 5)`

In [ ]:
# départ
print(b)

In [ ]:
# arrivée
print(c-a)

Vous commencez à mieux voir comment ça fonctionne; s'il existe une direction dans laquelle on peut "tirer" les données pour faire coincider les formes, on peut faire du braocasting. Et ça marche dans toutes les directions, comme on va le voir tout de suite.

### Broadcasting `(3, 5)` et `(3, 1)`

Au lieu d'ajouter à `a` une ligne, on peut lui ajouter une colonne, pourvu qu'elle ait la même taille que les colonnes de `a` :

In [ ]:
print(a)

In [ ]:
b = np.arange(1, 4)\
  .reshape(3, 1)
print(b)

****

Voyons comment se passe le broadcasting dans ce cas-là :

In [ ]:
c = a + b
print(c)

In [ ]:
print(c - a)

Vous voyez que tout se passe exactement que lorsqu'on avait ajouté une simple ligne, on a cette fois "tiré" la colonne dans la direction des lignes, pour passer :

depuis la dimension `(3, 1)`

vers la dimension `(3, 5)`

In [ ]:
# départ
print(b)

In [ ]:
# arrivée
print(c-a)

### Broadcasting `(3, 1)` et `(1, 5)`

Nous avons maintenant tous les éléments en main pour comprendre un exemple plus intéressant, où les deux tableaux ont des formes pas vraiment compatibles à première vue :

In [ ]:
col = b; print(b)

In [ ]:
line = 100 * np.arange(1, 6)
print(line)

****

Grâce au broadcasting, on peut additionner ces deux tableaux pour obtenir ceci : 

In [ ]:
m = col + line; print(m)

Remarquez qu'ici les **deux** entrées ont été étirées pour atteindre une dimension commune.

## Broadcasting - dimensions supérieures

Exemples de dimensions compatibles

```
A   (3d array):  15 x 3 x 5
B   (scalaire):           1
Res (3d array):  15 x 3 x 5
```

```
A   (3d array):  15 x 3 x 5
B   (3d array):  15 x 1 x 5
Res (3d array):  15 x 3 x 5
```

```
A   (3d array):  15 x 3 x 5
B   (2d array):       3 x 5
Res (3d array):  15 x 3 x 5
```

```
A   (3d array):  15 x 3 x 5
B   (2d array):       3 x 1
Res (3d array):  15 x 3 x 5
```

Exemples de dimensions **non compatibles**

``` 
A   (1d array):  3
B   (1d array):  4 
```

```
A      (2d array):      2 x 1
B      (3d array):  8 x 4 x 3 
```

* On ne peut broadcaster que de **1** vers n
* si $p>1$ divise n, on ne **peut pas** broadcaster de p vers n 